In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# FOR PHASE 1 of Training
def lr_schedule_1(epoch): 
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

# FOR PHASE 2 of Training
def lr_schedule_2(epoch):  
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    elif epoch > 45:
        lrate = 0.0003        
    return lrate

# FOR PHASE 3 of Training
def lr_schedule_3(epoch): 
    lrate = 0.001
    #if epoch > 15:
    #    lrate = 0.0005
    #elif epoch > 25:
    #    lrate = 0.0003        
    return lrate
    
#Data normalization as per CIFAR-10 dataset instructions - https://www.cs.toronto.edu/~kriz/cifar.html
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)






170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
# Construct CNN

weight_decay = 1e-4
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# Add softmax (output) layer
model.add(Dense(num_classes, activation='softmax'))

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Only one model.fit call of the 3 listed below must be uncommented while execution

# For Phase 1: 125 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_1)])

# For Phase 2: 60 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=60,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_2)])

# For Phase 3: 30 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
y_pred_test = model.predict(x_test)
#y_pred_test
joint_prob = np.zeros((10,10)) #for CIFAR 10 (i,j) - state (true label) i, action(prediction prob) j
for i in range(10):
  idx  = np.where(np.argmax(y_test,1)==i)
  for j in range(10):
    joint_prob[i,j] = np.mean(y_pred_test[idx,j])

joint_prob  = joint_prob*0.1 # prior pmf over the set of true image classes in CIFAR-10  is uniform.

# joint_prob[x,a] is the joint probability that the true image class is 'x' and predicted image class is 'a'
joint_prob

array([[8.11042130e-02, 2.45032683e-03, 5.61587326e-03, 6.15321239e-04,
        2.62033213e-03, 9.38807323e-05, 8.83592106e-04, 1.55593632e-03,
        3.06428187e-03, 1.99624486e-03],
       [1.22838002e-03, 9.15833235e-02, 2.48852791e-04, 1.17362523e-04,
        1.23328157e-04, 6.61292055e-05, 7.24249147e-04, 2.96058459e-04,
        1.05826128e-03, 4.55405749e-03],
       [5.34598604e-03, 4.39985748e-04, 6.60717309e-02, 2.77450997e-03,
        8.36822540e-03, 2.99012493e-03, 8.17238614e-03, 4.69895005e-03,
        5.54322265e-04, 5.83783071e-04],
       [4.26296927e-03, 1.52458446e-03, 8.52613524e-03, 4.23382759e-02,
        9.40696523e-03, 1.07751772e-02, 1.22209340e-02, 7.84538686e-03,
        1.57496631e-03, 1.52460672e-03],
       [1.47406468e-03, 2.30105198e-04, 5.19651957e-03, 2.24952716e-03,
        7.52590239e-02, 1.77181512e-03, 6.49104789e-03, 6.72990233e-03,
        3.94574646e-04, 2.03419710e-04],
       [1.56731624e-03, 5.39446808e-04, 7.25622848e-03, 8.96089971e-03,
   

In [ ]:
#Code structure credits :  https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py